# Gradient Boosting
Gradient boosting is a method that goes through cycles to iteratively add models into an ensemble.

It begins by initializing the ensemble with a single model, whose predictions can be pretty naive. (Even if its
predictions are wildly inaccurate, subsequent additions to the ensemble will address those errors.)

Then, we start the cycle:

    1.  First, we use the current ensemble to generate predictions for each observation in the dataset. To make a
    prediction, we add the predictions from all models in the ensemble.
    2.  These predictions are used to calculate a loss function (like mean squared error, for instance).
    3.  Then, we use the loss function to fit a new model that will be added to the ensemble. Specifically, we
    determine model parameters so that adding this new model to the ensemble will reduce the loss.

    (Side note: The "gradient" in "gradient boosting" refers to the fact that we'll use gradient descent on the loss
    function to determine the parameters in the new model.)

    Finally, we add the new model to ensemble, and ...
    ... repeat!

XGBoost stands for extreme gradient boosting, which is an implementation of gradient boosting with several additional
features focused on performance and speed.

(Scikit-learn has another version of gradient boosting, but XGBoost has some
technical advantages.)

In [367]:
import pandas as pd
from pandas.api.types import is_numeric_dtype
from sklearn.model_selection import train_test_split, cross_val_score
from xgboost import XGBRegressor
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

In [368]:
df = pd.read_csv('data/Melbourne_housing_FULL.csv')
df.dropna(axis=0, subset=['Price'], inplace=True)

In [369]:
X_full = df.drop(columns='Price', axis=1, inplace=False)

In [370]:
numerical_columns = [col for col in X_full.columns if is_numeric_dtype(X_full[col])]
categorical_columns = [col for col in X_full.columns if not is_numeric_dtype(X_full[col])]
low_cardinality_columns = [col for col in X_full[categorical_columns].columns if X_full[col].nunique() <= 15]
feature_columns = numerical_columns + categorical_columns
target_column = 'Price'

In [371]:
X = X_full[feature_columns]
y = df[target_column]

In [372]:
numerical_transformer = SimpleImputer(strategy='constant')
categorical_transformer = Pipeline(
    steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('one_hot_encoder', OneHotEncoder(handle_unknown='ignore', sparse=False))
    ]
)
preprocessor = ColumnTransformer(
    transformers=[
        ('numerical', numerical_transformer, numerical_columns),
        ('categorical', categorical_transformer, low_cardinality_columns)
    ]
)

In [373]:
my_pipeline = Pipeline(
    steps=[
        ('preprocessor', preprocessor),
        ('model', XGBRegressor(n_estimators=1000, learning_rate=0.01, n_jobs=-1))
    ]
)

In [374]:
my_pipeline.fit(X, y)

[13:31:52] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('numerical',
                                                  SimpleImputer(strategy='constant'),
                                                  ['Rooms', 'Distance',
                                                   'Postcode', 'Bedroom2',
                                                   'Bathroom', 'Car',
                                                   'Landsize', 'BuildingArea',
                                                   'YearBuilt', 'Lattitude',
                                                   'Longtitude',
                                                   'Propertycount']),
                                                 ('categorical',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                          

In [375]:
y_pred = my_pipeline.predict(X)

In [376]:
score = mean_absolute_error(y, y_pred)
print('Mean absolute error using Pipeline on full X, y sets:\n', score)

Mean absolute error using Pipeline on full X, y sets:
 189114.38746352808


In [377]:
scores_KFold = (-1) * cross_val_score(
    estimator=my_pipeline,
    X=X,
    y=y,
    cv=5,
    scoring='neg_mean_absolute_error'
)

[13:32:00] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:32:08] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:32:16] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:32:23] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:32:35] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [378]:
print('Mean absolute error using Cross-validation:\n', scores_KFold.mean())

Mean absolute error using Cross-validation:
 198830.23666186456


In [379]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=22)

In [380]:
num_imputer = SimpleImputer(strategy='constant')
cat_imputer = SimpleImputer(strategy='most_frequent')
oh_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)

In [381]:
imputed_num_X_train = pd.DataFrame(
    num_imputer.fit_transform(X_train[numerical_columns]),
    columns=numerical_columns)

imputed_cat_X_train = pd.DataFrame(
    cat_imputer.fit_transform(X_train[low_cardinality_columns]),
    columns=low_cardinality_columns)

oh_train_cols = pd.DataFrame(
    oh_encoder.fit_transform(imputed_cat_X_train),
    index=imputed_cat_X_train.index)

In [382]:
imputed_num_X_test = pd.DataFrame(
    num_imputer.transform(X_test[numerical_columns]),
    columns=numerical_columns)

imputed_cat_X_test = pd.DataFrame(
    cat_imputer.transform(X_test[low_cardinality_columns]),
    columns=low_cardinality_columns
)

oh_test_cols = pd.DataFrame(
    oh_encoder.transform(imputed_cat_X_test),
    index=imputed_cat_X_test.index)

In [383]:
preprocessed_X_train = pd.concat([imputed_num_X_train, oh_train_cols], axis=1)
preprocessed_X_test = pd.concat([imputed_num_X_test, oh_test_cols], axis=1)

In [384]:
xgboost_model = XGBRegressor()

In [385]:
xgboost_model.fit(preprocessed_X_train, y_train)
new_y_pred = xgboost_model.predict(preprocessed_X_test)

[13:32:49] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [386]:
new_score = mean_absolute_error(y_test, new_y_pred)
print('Mean Absolute Error without Pipeline, single epoch:\n', new_score)

Mean Absolute Error without Pipeline, single epoch:
 193102.73758165736


# Parameter Tuning
· n_estimators

    specifies how many times to go through the modeling cycle described above. It is equal to the number of models that
    we include in the ensemble.

    - Too low value causes underfitting, which leads to inaccurate predictions on both training data and test data.
    - Too high value causes overfitting, which causes accurate predictions on training data, but inaccurate predictions
    on test data (which is what we care about).

    Typical values range from 100-1000, though this depends a lot on the learning_rate parameter discussed below.

        model = XGBRegressor(n_estimators=500)
        model.fit(X_train, y_train)

· early_stopping_rounds

    early_stopping_rounds offers a way to automatically find the ideal value for n_estimators. Early stopping causes
    the model to stop iterating when the validation score stops improving, even if we aren't at the hard stop for
    n_estimators.

    It's smart to set a high value for n_estimators and then use early_stopping_rounds to find the optimal time to stop
    iterating.

    Since random chance sometimes causes a single round where validation scores don't improve, you need to specify a
    number for how many rounds of straight deterioration to allow before stopping. Setting early_stopping_rounds=5 is a
    reasonable choice. In this case, we stop after 5 straight rounds of deteriorating validation scores.

    When using early_stopping_rounds, you also need to set aside some data for calculating the validation scores - this
    is done by setting the eval_set parameter.

        model = XGBRegressor(n_estimators=500)
        model.fit(X_train, y_train,
                  early_stopping_rounds=5,
                  eval_set=[(X_test, y_test)],
                  verbose=False)

· learning_rate

    Instead of getting predictions by simply adding up the predictions from each component model, we can multiply the
    predictions from each model by a small number (known as the learning rate) before adding them in.

    This means each tree we add to the ensemble helps us less. So, we can set a higher value for n_estimators without
    overfitting. If we use early stopping, the appropriate number of trees will be determined automatically.

    In general, a small learning rate and large number of estimators will yield more accurate XGBoost models, though it
    will also take the model longer to train since it does more iterations through the cycle. As default, XGBoost sets
    learning_rate=0.1.

        model = XGBRegressor(n_estimators=1000, learning_rate=0.01)
        model.fit(X_train, y_test,
                  early_stopping_rounds=5,
                  eval_set=[(X_test, y_test)],
                  verbose=False)

· n_jobs

    On larger datasets where runtime is a consideration, you can use parallelism to build your models faster. It's
    common to set the parameter n_jobs equal to the number of cores on your machine. On smaller datasets, this won't
    help.

    The resulting model won't be any better, so micro-optimizing for fitting time is typically nothing but a
    distraction. But, it's useful in large datasets where you would otherwise spend a long time waiting during the fit
    command.

        model = XGBRegressor(n_estimators=1000, learning_rate=0.01, n_jobs=4)
        model.fit(X_train, y_train,
                  early_stopping_rounds=5,
                  eval_set=[(X_test, y_test)],
                  verbose=False)

# Conclusions
XGBoost is a the leading software library for working with standard tabular data (the type of data you store in Pandas
DataFrames, as opposed to more exotic types of data like images and videos). With careful parameter tuning, you can
train highly accurate models.


In [387]:
xgb_model = XGBRegressor(n_estimators=1000, learning_rate=0.1, n_jobs=4)

In [388]:
xgb_model.fit(preprocessed_X_train, y_train,
              early_stopping_rounds=5,
              eval_set=[(preprocessed_X_test, y_test)],
              verbose=False)

[13:32:52] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(n_estimators=1000, n_jobs=4)

In [389]:
xgb_preds = xgb_model.predict(preprocessed_X_test)

In [391]:
xgb_score = mean_absolute_error(y_test, xgb_preds)
print('Score for XGBRegressor:\n', xgb_score)

Score for XGBRegressor:
 184049.9769386744
